In [1]:
#author: Amelie Bauerdick
#Wabnitz Lab

# Import Packages

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.colors import BoundaryNorm, ListedColormap
import seaborn as sns
import umap
import pandas_bokeh
from sklearn.preprocessing import QuantileTransformer
 #Bokeh
from bokeh.io import show
from bokeh.models import Div, HoverTool, CustomJS, ColumnDataSource, Slider, Range1d,LinearColorMapper, DataTable, DateFormatter, TableColumn, BooleanFilter, CDSView, CategoricalColorMapper
from bokeh.palettes import RdBu3,all_palettes
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import column, row
from bokeh.transform import factor_cmap

output_notebook()

Loading BokehJS ...

# Load data 

In [ ]:
data=pd.read_csv("path/DMAP_2.csv")
data

In [4]:
x= data[['x']].copy()
y= data[['y']].copy()
cluster= data[['cluster']].copy()

In [ ]:
list(data.columns)

# Plot Cluster

In [6]:
datasum=data[['cluster','file_number','donor']].copy()

In [ ]:
grouped_df = datasum.groupby(['cluster','condition','donor'])['donor'].count()
grouped_df.reset_index(name="count")

In [ ]:
grouped_df.to_csv('path/data.csv')

# Most Important Features

In [ ]:
datafinal=data.copy()
datafinal=datafinal[[     
                     'feature_1'
                   ]]
datafinal

In [ ]:
list(datafinal.columns)

# Rename Features

In [ ]:
columns_labels = { 
                    'feature_1':'feature 1',   
                 }

datafinal.rename(columns= columns_labels, inplace=True)
datafinal

# Correlation Matrix

In [ ]:
sns.set(font_scale=2.5)
plt.subplots(figsize=(25,20))

s=sns.heatmap(datafinal.corr(), annot = True,cmap='rocket')
s.set_yticklabels(labels=datafinal.corr(), size = 40)
s.set_xticklabels(labels=datafinal.corr(), size = 40)
plt.xticks(rotation=45,ha='right')
plt.savefig('png/dmap_correlation.png',dpi=300, bbox_inches='tight')

# Heatmap

In [19]:
frames=[datafinal,cluster]
datafinal=pd.concat(frames, axis=1)

In [20]:
heatmap=datafinal.copy()
heatmapfinal=heatmap.groupby(['cluster']).mean()

In [ ]:
scaled_df = (heatmapfinal - heatmapfinal.min(axis=0))/(heatmapfinal.max(axis=0) - heatmapfinal.min(axis=0))
scaled_df

In [ ]:
sns.set(font_scale=2)
corr=scaled_df.head(124)
heat = sns.clustermap(scaled_df, 
                      figsize=(19, 10),
                      cmap='rocket', 
                      vmin=0, 
                      vmax=1,
                      xticklabels=True, 
                      yticklabels=True, 
                      annot=True,
                      fmt=".2f"
                     )
heat.ax_cbar.set_position((1, .2, .03, .2))
plt.setp(heat.ax_heatmap.yaxis.get_majorticklabels(), rotation=0) 
plt.setp(heat.ax_heatmap.xaxis.get_majorticklabels(), rotation=45, ha='right') 

plt.savefig('png/dmap_heatmap.png',dpi=300, bbox_inches='tight')
plt.show()

# Miniature DensMAPs 

In [23]:
def scaler(df):
    scaler = QuantileTransformer(output_distribution='uniform')
    for column in df.columns:
        scaled_column = scaler.fit_transform(df[[column]])
        df[column] = scaled_column.flatten() 
    return df

In [ ]:
scaler(datafinal)

In [25]:
def dmap(feature):

 source=ColumnDataSource(datafinal)

 cm = LinearColorMapper( palette='Plasma256',             
                         low = min(datafinal[feature]),
                         high = max(datafinal[feature]) 
                       )     


 tools_emb = ['save']

 dmap = figure(plot_width=500,
                  plot_height=500, 
                  tools=tools_emb,
                  title=feature
              )
    
 dmap.circle('x', 'y',
                size=1, 
                fill_color={"field":feature, "transform":cm},
                alpha=1, 
                line_alpha=1, 
                line_width=0.03, 
                source=source, 
                name="datafinal", 
               )

 dmap.x_range = Range1d(-10, 20)
 dmap.y_range = Range1d(-5, 20)
 dmap.title.align = 'center'
 dmap.grid.visible = False
 dmap.axis.visible = False
 dmap.outline_line_color= None
 dmap.border_fill_color = None

 dmap.title.text_font_size = '33pt'
    
 show(dmap)

In [ ]:
frames=[datafinal,x,y]
datafinal=pd.concat(frames, axis=1)
datafinal

In [ ]:
dmap('feature'),